# SETUP

## Loading Library

In [1]:
#install.packages("BART")
#install.packages("bartMachine")
#install.packages("doParallel")
library(BART)
library(caret)
library(tidyverse)
library(bartMachine)

Loading required package: nlme
Loading required package: nnet
Loading required package: survival
Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘caret’

The following object is masked from ‘package:survival’:

    cluster

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.1.1       ✔ purrr   0.3.2  
✔ tidyr   0.8.3       ✔ dplyr   0.8.0.1
✔ readr   1.3.1       ✔ stringr 1.4.0  
✔ tibble  2.1.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse() masks nlme::collapse()
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::lag()      masks stats::lag()
✖ purrr::lift()     masks caret::lift()
Loading require

In [2]:
#Include the parallel library. If the next line does not work, run install.packages(“parallel”) first
library(parallel)
library(doParallel)

# Use the detectCores() function to find the number of cores in system
n_cores <- detectCores()
print(n_cores)
cl <- makePSOCKcluster(4)
registerDoParallel(cl)
stopCluster(cl)

[1] 4


## Loading Data

In [3]:
happiness.df <- read_csv(file="datasets/world-happiness/happiness.csv")
colnames(happiness.df) <- make.names(colnames(happiness.df) %>%  tolower)

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_double(),
  Country = col_character(),
  Region = col_character(),
  `Happiness Rank` = col_double(),
  `Happiness Score` = col_double(),
  `Economy (GDP per Capita)` = col_double(),
  Family = col_double(),
  `Health (Life Expectancy)` = col_double(),
  Freedom = col_double(),
  `Trust (Government Corruption)` = col_double(),
  Generosity = col_double(),
  `Dystopia Residual` = col_double(),
  year = col_double(),
  nyears = col_double()
)


We can get the train-test split using createDataPartition from caret:

In [4]:
happiness.df <- happiness.df %>% select_if(is_double) %>% select(-c(x1,happiness.rank))

In [5]:
set.seed(42)
trainIndex <- createDataPartition(happiness.df$`happiness.score`, p = 0.9, list = FALSE)
happy.train <- happiness.df[ trainIndex,]
happy.test  <- happiness.df[-trainIndex,]

## bartMachine

In [6]:
registerDoParallel(cl)

Training: Run in 1hr 16 minutes on 4 cores:

In [8]:
set_bart_machine_num_cores(4)
happy.bm <- train(happiness.score ~ ., data = happy.train, 
                 method = "bartMachine", 
                 verbose = TRUE)

bartMachine now using 4 cores.
bartMachine initializing with 50 trees...
bartMachine vars checked...
bartMachine java init...
bartMachine factors created...
bartMachine before preprocess...
bartMachine after preprocess... 10 total features...
bartMachine sigsq estimated...
bartMachine training data finalized...
Now building bartMachine for regression ...
evaluating in sample data...done


In [ ]:
happy.bm <- loadRDS("model_files/bart_model_1.RDS")

In [20]:
summary(happy.bm)

bartMachine v1.2.3 for regression

training data n = 426 and p = 9 
built in 2.2 secs on 4 cores, 50 trees, 250 burn-in and 1000 post. samples

sigsq est for y beforehand: 0 
avg sigsq estimate after burn-in: 0.00522 

in-sample statistics:
 L1 = 11.39 
 L2 = 0.51 
 rmse = 0.03 
 Pseudo-Rsq = 0.9991
p-val for shapiro-wilk test of normality of residuals: 0.00131 
p-val for zero-mean noise: 0.93453 



In [14]:
saveRDS(happy.bm,file = "model_files/bart_model_1.RDS")

In [18]:
predict(happy.bm,newdata = happy.test)

[1] 6.882147 6.681631 6.609850 6.571304 6.447733 6.272347 5.845991 5.931919
 [9] 5.886691 5.191256 4.969359 4.908662 4.524494 7.020219 6.648026 6.571269
[17] 6.260682 6.002741 6.001113 5.808437 5.533708 5.367926 5.235063 5.240043
[25] 4.319588 4.348011 4.139963 4.176105 3.938163 3.924975 3.879452 3.328174
[33] 6.880499 5.792638 5.763854 5.552476 5.301196 4.867085 4.818335 4.755153
[41] 4.522498 4.294271 4.282734 3.703278

In [19]:
stopCluster(cl)

In [22]:
varImp(happy.bm)

.....


bartMachine variable importance

                              Overall
dystopia.residual             100.000
economy..gdp.per.capita.       63.081
family                         59.979
health..life.expectancy.       43.648
freedom                        35.538
generosity                     30.127
trust..government.corruption.  25.618
nyears                          1.962
year                            0.000